In [1]:
!pip3 freeze > requirements.txt 

# Import

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from src.preprocessing import normalize_csi
from src.csi_spliter import separate_amp_phase
from src.handle_csi_csv import split_csi_csv
from src.data import get_dataloaders, list_of_df_to_array
from src.model import TCN_AA
from src.train import train

import warnings
warnings.filterwarnings('ignore')

# Set Path

In [ ]:
path = os.path.abspath(".")
sys.path.insert(0, path) # location of src 

path

# Pre Process 

In [9]:
# 1. เตรียม path
csi_data_dir = os.path.join(path, 'csi_data')
keypoint_csv_dir = os.path.join(path, 'keypoint_csv')

# 2. อ่านไฟล์ feature และ label
feature_files = sorted(os.listdir(csi_data_dir))
label_files = sorted(os.listdir(keypoint_csv_dir))

X = []
Y = []

In [10]:

for feat_file, label_file in zip(feature_files, label_files):
    # อ่าน feature (สมมติเป็น .npy หรือ .csv)
    X_temp = []
    Y_temp = []
    feat_path = os.path.join(csi_data_dir, feat_file)
    for filename in os.listdir(feat_path):
        file_path = os.path.join(feat_path, filename)
        if file_path.endswith('.npy'):
            feat = np.load(file_path)
        elif file_path.endswith('.csv'):
            feat = pd.read_csv(file_path)
        else:
            continue  # ข้ามไฟล์ที่ไม่รู้จัก
        
        
                # อ่าน label (สมมติเป็น .csv)
        label_path = os.path.join(keypoint_csv_dir, label_file, filename)
        label = pd.read_csv(label_path)

        feat.set_index('timestamp', inplace=True)
        label.set_index('frame_index', inplace=True)    
        
        feat.reset_index(drop=True, inplace=True)
        feat.index += 1  # ให้ index เริ่มที่ 1

        X_temp.append(feat)  
        Y_temp.append(label)
    
    X.append(X_temp)
    Y.append(Y_temp)


In [11]:

X_norm_df_list = []
Y_norm_df_list = []

for i in range(len(X)): 
    for j in range(len(X[i])):
        X_temp = pd.DataFrame(X[i][j])
        Y_temp = pd.DataFrame(Y[i][j])
        
        X_temp = split_csi_csv(X_temp)

        aligned = X_temp.merge(Y_temp, left_index=True, right_index=True)

        if aligned.empty == True:
            continue

        # 2. แยก X, Y ที่ align แล้ว  
        X_aligned = aligned[X_temp.columns]
        Y_aligned = aligned[Y_temp.columns]

        # 3. Normalize (ยกเว้น timestamp)
        X_norm = normalize_csi(X_aligned)
        Y_norm = normalize_csi(Y_aligned)

        # 4. ถ้าต้องการ DataFrame กลับมา
        X_norm_df = pd.DataFrame(X_norm, columns=X_aligned.columns)
        Y_norm_df = pd.DataFrame(Y_norm, columns=Y_aligned.columns)

        X_norm_df_list.append(X_norm_df)
        Y_norm_df_list.append(Y_norm_df)
        # X_norm_df.to_csv("X_norm.csv")
        # Y_norm_df.to_csv("Y_norm.csv")
        # aligned.to_csv("aligned.csv")

# Train Model

In [ ]:
# for i, df in enumerate(X_norm_df_list):
#     print(f"X[{i}] shape: {df.shape}")
# for i, df in enumerate(Y_norm_df_list):
#     print(f"Y[{i}] shape: {df.shape}")
# 1. แบ่ง train/val (split ที่ list เลย)
X_train_list, X_val_list, Y_train_list, Y_val_list = train_test_split(
    X_norm_df_list, Y_norm_df_list, test_size=0.2, random_state=42
)

# 2. แปลง list ของ DataFrame เป็น numpy array
X_train, Y_train = list_of_df_to_array(X_train_list, Y_train_list)
X_val, Y_val = list_of_df_to_array(X_val_list, Y_val_list)

# 3. สร้าง DataLoader
train_loader, val_loader = get_dataloaders(X_train, Y_train, X_val, Y_val, batch_size=32)

# 4. สร้างและ train model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TCN_AA(num_inputs=X_train.shape[2], num_channels=[64, 64, 64], num_classes=Y_train.shape[2]).to(device)
# model = TCN_AA(num_inputs=X_train.shape[2], num_channels=[64, 64, 64], num_classes=Y_train.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

out = train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=30)
out